In [1]:
import os.path, gym
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import roboschool
import pdb
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d
GAMMA = .97
from functools import partial
NUM_HISTORY = N_HISTORY = 6
try:
    env.close()
except:
    pass
if 1:
    envname = "RoboschoolInvertedPendulum-v1"
    SIZE_MULT = 1
    REWARD_MULT = 1
    RANDOM_STYLE_COMPLEX = 0
    N_LAYERS = 10
else:
    envname = "RoboschoolHumanoidFlagrun-v1"
    SIZE_MULT = 4
    REWARD_MULT = 4
    RANDOM_STYLE_COMPLEX = 1
    N_LAYERS = 20
PERCENT_CHOOSE_OPTIMAL = .8
env = gym.make(envname)
N_OBS, N_ACT = [v.shape[0] for v in [env.observation_space, env.action_space]]
N_STATE = (N_OBS + N_ACT) * 2
REWARD_IN_STATE = 0
if REWARD_IN_STATE:
    N_STATE += 2
INPUT_UNITS = N_STATE * NUM_HISTORY
STATE_DECAY = .8
TESTING_GRAD_NORMS = 1
ADV_ENABLED = 0
STORED_MODELS = {}
FCNS = {'np':{
    'concat':np.concatenate,
    'reshape':np.reshape,
    'expand':np.expand_dims
},'tf':{
    'concat':tf.concat,
    'reshape':tf.reshape,
    'expand':tf.expand_dims
}}

/anaconda3/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [5]:

def apply_clipped_optimizer(opt_fcn,
                            loss,
                            clip_norm=.1,
                            clip_single=.03,
                            clip_global_norm=False,
                            var_list=None):
    if var_list is None:
        gvs = opt_fcn.compute_gradients(loss)
    else:
        gvs = opt_fcn.compute_gradients(loss, var_list = var_list)
        

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs],clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
                tf.reduce_sum([
                        tf.reduce_sum(tf.square(grad)) for grad, var in gvs
                        if grad is not None
                ]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                                    for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var)
                                    for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total

def MLP(x, lshapes, output_units, name_fcn, is_train, 
        self = None, usemodel = False, storemodel = False,
       scale = 1):
    if storemodel:
        #[[a_idx, c_idx, v_idx], name]
        modeltype = storemodel[0]
        STORED_MODELS[storemodel[1]] = [
            getattr(self, modeltype), lshapes, output_units, name_fcn, is_train]
    reuse = False
    if usemodel:
        modeltype = usemodel[0]
        existing_idx, lshapes, output_units, name_fcn, is_train = STORED_MODELS[usemodel[1]]
        old_idx = getattr(self, modeltype)
        setattr(self, modeltype, existing_idx)
        reuse = True
    h = [x]
    h.append(tf.nn.leaky_relu(tf.layers.dense(
        h[-1], lshapes[0], name=name_fcn(), reuse = reuse)))
    init = 0
    for size in lshapes:
        if not init:
            init = 1
            h2 = h[-1]
        h.append(tf.nn.leaky_relu(h[-1] + tf.layers.dense(
            h2, size, name=name_fcn(), reuse = reuse#,
#             kernel_initializer = tf.keras.initializers.Orthogonal(gain=0.1))
        )))
        #h[-1] = tf.layers.batch_normalization(h[-1], momentum = .9, training = is_train)
        h2 = tf.concat((x, h[-1]), -1)
    hout = tf.concat((h[len(h)//2:]), -1)
    output = tf.layers.dense(
        hout, output_units, name=name_fcn(), reuse = reuse) 
#         kernel_initializer = tf.keras.initializers.Orthogonal(gain=0.1),
#         bias_initializer = tf.keras.initializers.zeros())
    #output = tf.layers.batch_normalization(output, momentum = .9, training = is_train)
    if output_units == 1:
        output = tf.squeeze(output, -1)
    if usemodel:
        setattr(self, modeltype, old_idx)
    return output, leaky_tanh(output) * scale

In [6]:


def make_state(state, action, style = 'np'):
    together = FCNS[style]['concat']((state, action), -1)
    if style == 'tf':
        return together
#         expanded = FCNS[style]['expand'](
#             together, 1)
    else:
        expanded = FCNS[style]['expand'](
            together, 0)
    return expanded
    return FCNS[style]['reshape'](
        together, (state.shape[0], -1, state.shape[-1] + action.shape[-1]))
        
    

def accumulate_state(state, action, old_state, statedecay, style = 'np'):
    new_state = make_state(state, action, style)
    new_state_len = new_state.shape[-1]
    if style == 'tf':
        vel = new_state - old_state[:,:,0,:new_state_len]
    else:
        vel = new_state - old_state[0,:new_state_len]
    new_state = FCNS[style]['concat']((new_state, vel), -1)
    if style == 'tf':
        return FCNS[style]['concat']((tf.expand_dims(new_state, 2), old_state[:,:,:-1,:]*statedecay), 2)
    return FCNS[style]['concat']((new_state, old_state[:-1,:]*statedecay), 0)

def leaky_tanh(x):
    return tf.nn.tanh(x*30)/10 + tf.nn.tanh(x*2)/2 + tf.nn.tanh(x/10) * 2 + x * 1e-3

In [ ]:


class PolicyLearner(object):
    def __init__(self, ob_space, ac_space, take_weights_here=None, 
                 lshapes = [64 * SIZE_MULT] * N_LAYERS, config = None, 
                 lshapes_small = [32 * SIZE_MULT] * N_LAYERS,
                 lshapes_smaller = [32 * SIZE_MULT] * N_LAYERS):
        self.a_idx = 0
        self.c_idx = 0
        self.m_idx = 0
        self.v_idx = 0
#         self.vraw_idx = 0
        self.sess = tf.InteractiveSession(config=config)
        self.obs_raw = tf.placeholder(tf.float32, (None, None, NUM_HISTORY, N_STATE))
        self.returns = tf.placeholder(tf.float32, (None, None))
        self.returnsdecayed = tf.placeholder(tf.float32, (None, None))
        self.mask = tf.placeholder(tf.float32, (None, None))
        self.lr = tf.placeholder_with_default(1e-3, (None))
        self.statesraw = tf.placeholder(tf.float32, (None, None, N_OBS))
        self.is_train = tf.placeholder_with_default(True, (None))
        self.obs = tf.reshape(
            self.obs_raw, (
                tf.shape(self.obs_raw)[0], tf.shape(self.obs_raw)[1], N_STATE * NUM_HISTORY))
        
        self.raw_actions, self.actions = MLP(
            self.obs, lshapes, N_ACT, self.a_name, self.is_train, self = self,
            storemodel = ['a_idx', 'action'])
        if len(self.actions.shape) == 2:
            self.actions = tf.expand_dims(self.actions, -1)
        
        _, self.state_value_estimate = MLP(
            self.obs, lshapes_smaller, 1, self.v_name, self.is_train, self = self,
            storemodel = ['v_idx', 'state_value'], scale = 2 * REWARD_MULT)
        
        self.advantage = ((
            self.state_value_estimate[:,1:] * GAMMA + self.returns[:,:-1]) -
            self.state_value_estimate[:,:-1])
        
        self.critic_input = tf.concat((self.obs, self.actions), -1)
        
        _, self.advantage_estimator = MLP(
            self.critic_input, lshapes_small, 1, self.c_name, self.is_train, self = self,
            storemodel = ['c_idx', 'critic'], scale = 5 * REWARD_MULT)
        _, self.model_state_estimate = MLP(
            self.critic_input, lshapes, N_OBS, self.m_name, self.is_train)
        self.future_obs_raw = accumulate_state(
            self.model_state_estimate, self.actions, self.obs_raw, STATE_DECAY, style = 'tf')
        
        self.future_obs = tf.reshape(self.future_obs_raw, 
            (tf.shape(self.future_obs_raw)[0],tf.shape(self.future_obs_raw)[1], N_STATE * NUM_HISTORY))
        
        _, self.model_state_value = MLP(
            self.future_obs, lshapes_smaller, 1, 
            self.v_name, self.is_train, self = self,
            usemodel = ['v_idx', 'state_value'], scale = 2 * REWARD_MULT)
        

        _, self.future_actions = MLP(
            self.future_obs, lshapes, N_ACT, self.a_name, self.is_train, self = self,
            usemodel = ['a_idx', 'action'])
        if len(self.future_actions.shape) == 2:
            self.future_actions = tf.expand_dims(self.future_actions, -1)
        
        self.future_critic_input = tf.concat((self.future_obs, self.future_actions), -1)
        _, self.future_advantage_estimator = MLP(
            self.future_critic_input, lshapes_small, 1, self.c_name, self.is_train, self = self,
            usemodel = ['c_idx', 'critic'], scale = 5 * REWARD_MULT)
        
        if len(self.future_actions.shape) == 2:
            self.future_actions = tf.expand_dims(self.future_actions, -1)
        
        self.t_vars = tf.trainable_variables()
        self.a_vars = [var for var in self.t_vars if 'a_' in var.name]
        self.v_vars = [var for var in self.t_vars if 'v_' in var.name]
        self.c_vars = [var for var in self.t_vars if 'c_' in var.name]
        self.m_vars = [var for var in self.t_vars if 'm_' in var.name]
        
        self.creg, self.areg, self.vreg, self.mreg = [
            tf.reduce_mean([tf.reduce_mean(tf.square(v)) for v in optvars]) * 1e0
            for optvars in 
            [self.c_vars, self.a_vars, self.v_vars, self.m_vars]]
        self.maskexpanded = tf.expand_dims(self.mask, -1)
        
        self.m_loss_raw = tf.reduce_mean(tf.square(
            self.model_state_estimate[:,:-1] - self.statesraw[:,1:]) * self.maskexpanded[:,:-1]) * 100
        self.m_loss = self.m_loss_raw + self.mreg
        
        self.v_loss_raw = tf.reduce_mean(tf.square(
            self.returnsdecayed - self.state_value_estimate) * self.mask)
        self.v_loss =  self.v_loss_raw + self.vreg
        
        self.c_loss_raw = tf.reduce_mean(tf.square(
            self.advantage_estimator[:,:-1] - self.advantage) * self.mask[:,:-1])
        self.c_loss =  self.c_loss_raw + self.creg
        
        
        self.actionmean = tf.reduce_sum(
            self.actions * self.maskexpanded, 1) / tf.reduce_sum(
            self.maskexpanded, 1) + tf.reduce_sum(
            self.future_actions * self.maskexpanded, 1) / tf.reduce_sum(
            self.maskexpanded, 1) 
        
        
        self.actionmean = tf.reduce_sum(
            self.actions * self.maskexpanded, 1) / tf.reduce_sum(
            self.maskexpanded, 1)
        self.actdiffsquared = tf.square(
            self.actions - tf.expand_dims(self.actionmean, 1))
        self.actvar = tf.reduce_sum(
            self.actdiffsquared * self.maskexpanded, 1)/ tf.reduce_sum(
            self.maskexpanded, 1)
        self.actstd = self.actvar
        self.actstdpenalty = -tf.reduce_mean(tf.log(self.actstd + .00001) + tf.log(1.00001 - self.actstd)) * .001
        
        self.aregmeanbyaction = tf.reduce_mean(
            tf.square(self.actionmean)) * .01
        self.a_mse = tf.reduce_mean(tf.square(
            self.actions + self.future_actions + self.raw_actions)) * 1e-2
        self.aregtotal = self.areg + self.aregmeanbyaction + self.actstdpenalty + self.a_mse
        
        self.frac_not_masked = tf.reduce_mean(self.mask)
        
        self.a_loss_critic = -tf.reduce_mean(
            self.advantage_estimator * self.mask)/self.frac_not_masked * 3
        self.a_loss_model = -tf.reduce_mean(
            self.model_state_value * self.mask)/self.frac_not_masked * 8
        self.a_loss_secondaction = -tf.reduce_mean(
            self.future_advantage_estimator * self.mask)/self.frac_not_masked * 20
        self.a_loss_raw = self.a_loss_critic + self.a_loss_model + \
            self.a_loss_secondaction
        
        self.a_loss = self.a_loss_raw + self.aregtotal
        
        self.grad_v = tf.square(tf.gradients(self.state_value_estimate * self.mask, self.obs)[0])
        self.grad_m = tf.square(tf.gradients(self.model_state_estimate * self.maskexpanded, self.critic_input)[0])
        self.grad_c = tf.square(tf.gradients(self.advantage_estimator * self.mask, self.critic_input)[0])
        self.grad_norm_m = tf.reduce_mean(self.grad_m) * 1e1
        self.grad_norm_v = tf.reduce_mean(self.grad_v) * 1e0
        self.grad_norm_c = tf.reduce_mean(self.grad_c) * 1e1
    
        slopes = tf.reduce_sum(tf.square(self.grad_c), reduction_indices=[2])
        self.grad_c_1 = tf.reduce_mean(self.mask * (slopes - .1) ** 2) * 1e2
        slopes = tf.reduce_sum(tf.square(self.grad_m), reduction_indices=[2])
        self.grad_m_1 = tf.reduce_mean(self.mask * (slopes - .1) ** 2) * 1e2
       
        self.a_loss_minimize = self.a_loss
        self.c_loss_minimize = self.c_loss + self.grad_norm_c + self.grad_c_1
        self.v_loss_minimize = self.v_loss + self.grad_norm_v
        self.m_loss_minimize = self.m_loss + self.grad_norm_m + self.grad_m_1
        
        self.critic_opt = tf.train.AdamOptimizer(self.lr/3, beta1= .8)
        self.value_opt = tf.train.AdamOptimizer(self.lr/3, beta1= .8)
        self.actor_opt = tf.train.AdamOptimizer(self.lr, beta1= .8)
        self.model_opt = tf.train.AdamOptimizer(self.lr/3, beta1= .8)
        
        def get_grad_norm(optimizer, loss):
            gvs = optimizer.compute_gradients(loss)
            grad_norm = tf.reduce_mean(
                [tf.reduce_mean(tf.square(grad)) for
                 grad, var in gvs if grad is not None])
            return grad_norm
        
        if TESTING_GRAD_NORMS:
            self.a_grads = [
                get_grad_norm(self.actor_opt, l) for l in [
                self.a_loss_minimize, self.a_loss_raw, self.a_loss_critic,self.a_loss_model,
                self.a_loss_secondaction,
                self.areg, self.aregmeanbyaction, self.actstdpenalty
            ]]
            self.v_grads = [get_grad_norm(self.value_opt, l) for l in [
                self.v_loss_minimize, self.v_loss_raw, 
                self.vreg, self.grad_norm_v
            ]]
            self.c_grads = [get_grad_norm(self.critic_opt, l) for l in [
                self.c_loss_minimize, self.c_loss_raw, 
                self.creg, self.grad_norm_c, self.grad_c_1

            ]]
            self.m_grads = [get_grad_norm(self.critic_opt, l) for l in [
                self.m_loss_minimize, self.m_loss_raw, 
                self.mreg, self.grad_norm_m, self.grad_m_1
            ]]
        
        self.copt, self.c_norm = apply_clipped_optimizer(
            self.critic_opt, self.c_loss_minimize, var_list = self.c_vars)
        self.vopt, self.v_norm = apply_clipped_optimizer(
            self.value_opt, self.v_loss_minimize, var_list = self.v_vars)
        self.aopt, self.a_norm = apply_clipped_optimizer(
            self.actor_opt, self.a_loss_minimize, var_list = self.a_vars)
        self.mopt, self.m_norm = apply_clipped_optimizer(
            self.model_opt, self.m_loss_minimize, var_list = self.m_vars)

    def a_name(self):
        self.a_idx += 1
        return 'a_' + str(self.a_idx)
    def c_name(self):
        self.c_idx += 1
        return 'c_' + str(self.c_idx)
    def v_name(self):
        self.v_idx += 1
        return 'v_' + str(self.v_idx)
    def m_name(self):
        self.m_idx += 1
        return 'm_' + str(self.m_idx)
    
    def load_weights(self):
        feed_dict = {}
        for (var, w), ph in zip(
            self.assigns, self.weight_assignment_placeholders):
            feed_dict[ph] = w
        self.sess.run(self.weight_assignment_nodes, feed_dict=feed_dict)

    def act(self, obs):
        # Because we need batch dimension, 
        # data[None] changes shape from [A] to [1,A]
        a = self.sess.run(
            self.actions, feed_dict={
                self.obs_raw:np.reshape(obs, (1, 1, N_HISTORY, N_STATE)),
                self.is_train:False
            })
        return a[0][0]  # return first in batch



config = tf.ConfigProto(
    inter_op_parallelism_threads=0,
    intra_op_parallelism_threads=0,
    device_count = { "GPU": 0 } )
tf.reset_default_graph()

pi = PolicyLearner(env.observation_space, env.action_space, config = config)

sess = pi.sess
self = pi
sess.run(tf.global_variables_initializer())
#trainer = ZooPolicyTensorflow(
# "mymodel1", env.observation_space, env.action_space)
saver = tf.train.Saver()

/anaconda3/envs/py35/lib/python3.5/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
# env.close()


# env = gym.make(envname)

In [ ]:

#         self.advantage = ((
#             self.stateraw_value_estimate[:,1:] * GAMMA + self.returns) -
#             self.state_value_estimate[:,:-1])

In [ ]:

ah, sh, shraw, rh, rdecayedh, maskh = [
    np.zeros((0, 0, i)) for i in [N_ACT, INPUT_UNITS, N_OBS, 1, 1, 1]]
globalframes = []
localframes = []
ep = 0
trained = 0
ongoing = 0
printfreq = 10
obj_fname = envname + str(PERCENT_CHOOSE_OPTIMAL) + 'saveobjs_unguided.pkl'
tffile = "tmp/" + envname + str(PERCENT_CHOOSE_OPTIMAL) + "unguided_trained.ckpt"
import pickle

In [ ]:
ongoing = 0
sess.run(tf.global_variables_initializer())

In [ ]:
if ongoing:
    
    save_path = saver.save(sess, tffile)
    print('saved at epoch', ep)
    with open(obj_fname,"wb") as f:
        pickle.dump(
            [ah, sh, rh, rdecayedh, maskh, ep, globalframes
            ], f)
    trained = 1

In [ ]:
trained = 0

In [ ]:
if trained:
    try:
        saver.restore(sess, tffile)
        with open(obj_fname, "rb") as f:
            ah, sh, shraw, rh, rdecayedh, maskh, ep, globalframes = pickle.load(f)
        print('restored from save file')
    except:
        print('no save file detected')
        trained = 0
MAX_SEQ_LEN = 5000
for ep in range(ep, 10000000):
    if ep % 100 == 0 and trained and ep > 0:
        save_path = saver.save(sess, tffile)
        print('saved at epoch', ep)
        with open(obj_fname,"wb") as f:
            pickle.dump(
                [ah[-1000:], sh[-1000:], shraw[-1000:],
                 rh[-1000:], rdecayedh[-1000:], maskh[-1000:], ep, globalframes
                ], f)
    trained = 1
    ongoing = 1
    
    an, sn, snraw, rn, rdecayedn, maskn = [
        np.zeros((0, i)) for i in [N_ACT, INPUT_UNITS, N_OBS, 1, 1, 1]]
    frame = 0
    score = 0
    restart_delay = 5
    obs = env.reset()
    obsraw = obs
    snraw = np.concatenate((snraw, obs.reshape(1, -1)), 0)
    obs = np.concatenate((obs, np.zeros(N_ACT)))
    obs = np.concatenate((obs, np.zeros_like(obs)))
    obs_mat = np.concatenate((
        obs[None,:],np.zeros((NUM_HISTORY-1, N_STATE))), 0)
    rn = [0]
    done_ctr = 0
    sn = np.concatenate((sn, obs_mat.reshape(1, -1)), 0)
    step_num = 0
    step_shifted = -1
    show_shift = 0
    if np.random.rand() > .5:
        prob_random = PERCENT_CHOOSE_OPTIMAL#2#.95#1 - 1/np.sqrt(ep+1)
    else:
        prob_random = 2
    while 1:
        step_num += 1
        a = pi.act(obs_mat.flatten())
        if step_shifted == -1: #always act on your own
            shifted = 1
            if np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                a = np.random.randn(*a.shape)*.01
            elif np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                a = a + np.random.randn(*a.shape) * .5
                #a = a + np.random.randn(*a.shape)/np.sqrt(np.sqrt(ep + 2))*4
            elif np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                impact = np.random.binomial(size=a.shape, n=1, p= 0.5) * 2
                if np.random.rand() > .5:
                    a = a + impact
                else:
                    a = a - impact
            elif np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                impact = np.random.binomial(size=a.shape, n=1, p= 0.5) * 2 - 1
                a = a + impact
            elif np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                a = a * .5
            elif np.random.rand() > prob_random and RANDOM_STYLE_COMPLEX:
                a = a * 2
            elif np.random.rand() > prob_random and not RANDOM_STYLE_COMPLEX:
                a = (np.random.binomial(size=a.shape, n=1, p= 0.5) * 2 - 1
                    ) * np.random.randn()
            else:
                shifted = 0
            if shifted:
                step_shifted = step_num
        an = np.concatenate((an, a[None,:]), 0)
        snraw = np.concatenate((snraw, obsraw[None,:]), 0)
        last_obs = obs
#         last_obsraw = obsraw
        obs, r, done, _ = env.step(a)
        obsraw = obs
        r = r + 1 * REWARD_MULT

        obs_mat = accumulate_state(
            obs, a, obs_mat, STATE_DECAY, style = 'np')
        
        rn.append(r)
        sn = np.concatenate((sn, obs_mat.reshape(1, -1)), 0)
        score += r
        frame += 1
        still_open = env.render("human")
        if done:
            done_ctr += 1
            if done_ctr > 3:
                if ep % MAX_SEQ_LEN == 0:
                    print('score', score, ' frames', frame)
                break
        if still_open==False:
            crashhere
        if not done: continue
        if restart_delay==0:
            print("score=%0.2f in %i frames" % (score, frame))
            if still_open!=True:      # not True in multiplayer or non-Roboschool 
                break
            restart_delay = 2000*2  # 2 sec at 60 fps
        restart_delay -= 1
        if restart_delay==0: 
            break
    a = pi.act(obs_mat.flatten())
    an = np.concatenate((an, a[None,:]), 0)
    localframes.append(frame)
    rn = np.array(rn)
    rn[-1] = rn[-1] - 20 * REWARD_MULT
    rewards = [0]
    for ir in rn[::-1]:
        rewards.append(rewards[-1] * GAMMA + ir)
    rdecayedn = np.array(rewards)[:0:-1]
    lenrdec = len(rdecayedn)
#     rdecayedn = rdecayedn + lenrdec
#     rdecayedn = rdecayedn - np.arange(lenrdec) * 1.8
    maskn = np.ones_like(rn)
    if step_shifted > -1:
        if step_shifted == 0:
            raise ValueError('step shifted not allowed 0')
        maskn[:step_shifted - 1] = 0
    if ep == 0:
        ah, sh, shraw, rh, rdecayedh, maskh = [
            np.expand_dims(v, 0) for v in [an, sn, snraw, rn,rdecayedn, maskn]]
    else:
        def get_updated_h(h, n, third_dim):
            hshape = h.shape[1]
            nshape = n.shape[0]
            if third_dim:
                if hshape > nshape:
                    n = np.concatenate((n, np.zeros((hshape - nshape, n.shape[-1]))), 0)
                if nshape > hshape:
                    h = np.concatenate((h, np.zeros((
                        h.shape[0], nshape - hshape, h.shape[-1]))), 1)
            else:
                if hshape > nshape:
                    n = np.concatenate((n, np.zeros((hshape - nshape))), 0)
                if nshape > hshape:
                    h = np.concatenate((h, np.zeros((h.shape[0], nshape - hshape))), 1)
            h = np.concatenate((h, np.expand_dims(n, 0)), 0)
            return h
            
        ah, sh, shraw = [get_updated_h(h, n, 1) for  h, n in zip(
            [ah, sh, shraw], [an, sn, snraw])]
        
        rh, rdecayedh, maskh = [
            get_updated_h(h, n, 0) for h, n in zip(
                [rh, rdecayedh, maskh], [rn, rdecayedn, maskn])]
        
    if ep % 1 == 0 and ep > 20:
        ah, sh, shraw, rh,rdecayedh, maskh = [
            v[-400:] for v in [ah, sh, shraw, rh,rdecayedh, maskh]]
        globalframes.append(np.mean(localframes))
        localframes = []
        batch_size = 16
        if ep < batch_size:
            batch_size = ep
        num_hist = ah.shape[0]
        for itr in range(1):
            if num_hist >  batch_size:
                forced_hist = 2
                #probability = np.arange(num_hist - forced_hist)
                probability = num_steps_per_run = (
                    maskh.shape[1] - maskh[:,::-1].argmax(1))[:-forced_hist]
                probability = np.square(probability)
                probability = probability / probability.sum()
                samples = np.concatenate((
                    np.random.choice(
                        num_hist - forced_hist, batch_size - forced_hist, 
                        replace=False, p=probability),
                    np.arange(
                        num_hist - forced_hist, num_hist)))
            else:
                samples = np.random.choice(num_hist, num_hist, replace=False)
            actions, states, statesraw, returns, returnsdecayed, mask = [
                v[samples] for v in [ah, sh, shraw, rh,rdecayedh, maskh]]
            feed_dict={
                        self.returns:returns,
                        self.statesraw: statesraw,
                        self.returnsdecayed:returnsdecayed,
                        self.lr: .01 / np.power(ep + 20, .4),
                        self.mask:mask,
                        self.obs_raw: states.reshape(*states.shape[:2], N_HISTORY, N_STATE)
            }
            _, aloss = sess.run(
                [self.aopt, self.a_loss],
                feed_dict = feed_dict
                    )
            if TESTING_GRAD_NORMS and ep % printfreq == 0:
                a_grad_norms = self.sess.run(self.a_grads, feed_dict)
                print('a grad norms', a_grad_norms)
            feed_dict[self.actions] = actions
            
            _,_, _, mloss, closs, vloss = sess.run(
                [self.mopt, self.copt,self.vopt, self.m_loss, self.c_loss, self.v_loss],
                    feed_dict=feed_dict)
            if 1:
                if ep % printfreq == 0:
                    print('aloss', aloss, 'closs', closs, 'vloss', vloss)
                    if TESTING_GRAD_NORMS:
                        v_grad_norms, c_grad_norms, m_grad_norms = self.sess.run(
                            [self.v_grads, self.c_grads, self.m_grads], feed_dict)
                        print('v grad norms', v_grad_norms)
                        print('c grad norms', c_grad_norms)
                        print('m grad norms', m_grad_norms)
        if ep % printfreq == 0:
            print(' ep, ', ep, ' avg frames',np.mean(globalframes[-20:]))
            print('abs action',np.abs(ah)[-1,0,:].shape, np.abs(ah)[-1,0,:].mean())
            print('max reward',np.max(rh[-10:,10:]))
        if ep % 10000 == 0:
            clear_output()
        if ep % 100 == 0:
            ysmoothed = gaussian_filter1d(globalframes, sigma=4)
            plt.plot(ysmoothed)
            plt.show()


In [ ]:
ah[-1][10]

In [ ]:

            self.a_grads = [
                get_grad_norm(self.actor_opt, l) for l in [
                self.a_loss_minimize, self.a_loss_raw, self.a_loss_critic,self.a_loss_model,
                self.a_loss_secondaction,
                self.areg, self.aregmeanbyaction, self.actstdpenalty
            ]]
            self.v_grads = [get_grad_norm(self.value_opt, l) for l in [
                self.v_loss_minimize, self.v_loss_raw, 
                self.vreg, self.grad_norm_v
            ]]
            self.c_grads = [get_grad_norm(self.critic_opt, l) for l in [
                self.c_loss_minimize, self.c_loss_raw, 
                self.creg, self.grad_norm_c, self.grad_c_1

            ]]
            self.m_grads = [get_grad_norm(self.critic_opt, l) for l in [
                self.m_loss_minimize, self.m_loss_raw, 
                self.mreg, self.grad_norm_m, self.grad_m_1
            ]]

In [ ]:
actions.shape

In [ ]:
returns.shape, statesraw.shape, returnsdecayed.shape, mask.shape, \
    states.reshape(*states.shape[:2], N_HISTORY, N_STATE).shape

In [ ]:
self.sess.run(self.actions, feed_dict).shape

In [ ]:
self.sess.run(self.obs, feed_dict).shape

In [ ]:
self.sess.run(self.a_loss_secondaction, feed_dict).shape

In [ ]:
self.sess.run(self.future_actions, feed_dict).shape

In [ ]:
self.sess.run(self.maskexpanded, feed_dict).shape

In [ ]:

                        self.returns:returns,
                        self.statesraw: statesraw,
                        self.returnsdecayed:returnsdecayed,
                        self.lr: .01 / np.power(ep + 20, .4),
                        self.mask:mask,
                        self.obs_raw: states.reshape(*states.shape[:2], N_HISTORY, N_STATE)

In [ ]:
states.shape

In [ ]:
self.obs_raw

In [ ]:
a.shape, obs.shape, obs_mat.shape